# VAE

> Scripts to use Variational Autoencoders

In [8]:
#| default_exp vae

In [9]:
#| export
#| hide

import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torchmetrics import MetricCollection, MeanMetric

In [10]:
#| export
class BetaVAE(pl.LightningModule):
    def __init__(self, encoder, decoder, beta=1.0, loss_fn=None, optimizer_cls=torch.optim.Adam, lr=0.001, **kwargs):
        super().__init__(**kwargs)
        self.beta = beta
        self.encoder = encoder
        self.decoder = decoder
        self.loss_fn = loss_fn if loss_fn else self.default_loss_fn
        self.optimizer_cls = optimizer_cls
        self.lr = lr
        self.metrics = MetricCollection({
            'total_loss': MeanMetric(),
            'reconstruction_loss': MeanMetric(),
            'kl_loss': MeanMetric()
        })

    def forward(self, x):
        z_mean, z_log_var = self.encoder(x)
        z = self._reparameterize(z_mean, z_log_var)
        return self.decoder(z)

    def _reparameterize(self, mean, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mean + eps * std

    def default_loss_fn(self, x, x_hat, z_mean, z_log_var):
        # Calculate reconstruction loss between the original input x and the reconstructed input x_hat.
        reconst_loss = F.mse_loss(x_hat, x, reduction='mean')
        # Calculate the KL divergence to measure how much information is lost when using q(z|x) to represent p(z).
        kl_loss = -0.5 * torch.sum(1 + z_log_var - z_mean.pow(2) - z_log_var.exp(), dim=1).mean()
        return reconst_loss, self.beta * kl_loss

    def training_step(self, batch, batch_idx):
        x = batch
        z_mean, z_log_var = self.encoder(x)
        z = self._reparameterize(z_mean, z_log_var)
        x_hat = self.decoder(z)
        reconst_loss, kl_loss = self.loss_fn(x, x_hat, z_mean, z_log_var)
        total_loss = reconst_loss + kl_loss
        self.log('train_loss', total_loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return total_loss

    def configure_optimizers(self):
        return self.optimizer_cls(self.parameters(), lr=self.lr)

    def generate(self, n=1):
        z = torch.randn(n, self.latent_dim, device=self.device)
        return self.decoder(z)

In [11]:
#| hide
import nbdev; nbdev.nbdev_export()